In [1]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
# import librosa
# import librosa.display
# import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Model

In [2]:
df = pd.read_csv('../data/voice_data.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,label,emotions
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280941,-8.379515,-0.323866,...,-0.323118,-0.797864,-0.332338,0.177740,0.204333,-0.211895,0.080943,-0.274973,6,fearful
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,0.066087,0.029629,-0.052548,-0.059842,-0.084479,0.181888,0.202147,-0.165808,5,angry
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,0.057103,-0.174446,-0.116118,0.133232,0.191797,-0.322106,-0.287355,0.132173,8,surprise
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,0.015315,0.068387,0.201094,-0.062219,-0.291406,-0.284542,0.070689,0.088308,6,fearful
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193612,-8.719680,-7.731155,-8.692744,0.833810,...,-0.276208,-0.723615,0.074101,0.027505,0.288559,-0.022368,0.053056,0.346106,5,angry


In [3]:
df.drop(inplace=True, columns=["emotions"])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280941,-8.379515,-0.323866,...,0.428459,-0.323118,-0.797864,-0.332338,0.177740,0.204333,-0.211895,0.080943,-0.274973,6
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,-0.031505,0.066087,0.029629,-0.052548,-0.059842,-0.084479,0.181888,0.202147,-0.165808,5
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,0.236663,0.057103,-0.174446,-0.116118,0.133232,0.191797,-0.322106,-0.287355,0.132173,8
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,-0.129597,0.015315,0.068387,0.201094,-0.062219,-0.291406,-0.284542,0.070689,0.088308,6
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193612,-8.719680,-7.731155,-8.692744,0.833810,...,0.525122,-0.276208,-0.723615,0.074101,0.027505,0.288559,-0.022368,0.053056,0.346106,5


In [4]:
target = df["label"]

In [5]:
data = df.drop("label", axis=1)
feature_names = data.columns
data.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280941,-8.379515,-0.323866,...,-0.334692,0.428459,-0.323118,-0.797864,-0.332338,0.177740,0.204333,-0.211895,0.080943,-0.274973
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,-0.079512,-0.031505,0.066087,0.029629,-0.052548,-0.059842,-0.084479,0.181888,0.202147,-0.165808
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,-0.045710,0.236663,0.057103,-0.174446,-0.116118,0.133232,0.191797,-0.322106,-0.287355,0.132173
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,-0.175183,-0.129597,0.015315,0.068387,0.201094,-0.062219,-0.291406,-0.284542,0.070689,0.088308
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193612,-8.719680,-7.731155,-8.692744,0.833810,...,-0.403427,0.525122,-0.276208,-0.723615,0.074101,0.027505,0.288559,-0.022368,0.053056,0.346106


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
y_train

1611    5
1057    3
3033    6
1399    5
472     3
       ..
1130    3
1294    4
860     5
3507    5
3174    6
Name: label, Length: 2664, dtype: int64

In [8]:
from joblib import load

X_scaler = load('scaler.sav')

In [9]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=256, activation='relu', input_dim=128))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 2313      
Total params: 101,129
Trainable params: 101,129
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
84/84 - 0s - loss: 0.8509 - accuracy: 0.7012
Epoch 2/10
84/84 - 0s - loss: 0.4469 - accuracy: 0.8393
Epoch 3/10
84/84 - 0s - loss: 0.3319 - accuracy: 0.8878
Epoch 4/10
84/84 - 0s - loss: 0.2367 - accuracy: 0.9230
Epoch 5/10
84/84 - 0s - loss: 0.1668 - accuracy: 0.9508
Epoch 6/10
84/84 - 0s - loss: 0.1128 - accuracy: 0.9681
Epoch 7/10
84/84 - 0s - loss: 0.0709 - accuracy: 0.9854
Epoch 8/10
84/84 - 0s - loss: 0.0514 - accuracy: 0.9906
Epoch 9/10
84/84 - 0s - loss: 0.0285 - accuracy: 0.9981
Epoch 10/10
84/84 - 0s - loss: 0.0139 - accuracy: 0.9996


In [15]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions,))

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets